In [54]:
import sys
sys.path.append("..")
from common_utils import set_data_home
set_data_home("/mnt/d")
from common_utils import DATA_HOME, join
from lstm.sales_data import Sales_Dataset
from torch.utils.data.dataloader import DataLoader
import torch

SALE_HOME = join(DATA_HOME, "sales_data")
MODEL_HOME = join(DATA_HOME, "sale_model")
MODEL_PTH = join(MODEL_HOME, "sale_model.pth")


In [2]:
from torch.nn import LSTM
from torch.nn import MSELoss
from torch import optim
import matplotlib.pyplot as plt


I, H = 71, 33
epochs = 90

model = LSTM(71, 33, batch_first=True).cuda()
model.load_state_dict(torch.load(MODEL_PTH))
sd = Sales_Dataset(SALE_HOME, is_train=False)
dl = DataLoader(sd, batch_size=1)

In [3]:
import pandas as pd
ans_df = pd.DataFrame()
f_dict = sorted(list(sd.family_set))
h, c = torch.randn(1,1, 33, dtype=torch.float32).cuda(), torch.randn(1,1, 33, dtype=torch.float32).cuda()

for X, y in dl:
    yhat, (_, _) = model(X, (h, c))
    yhat = yhat[0]
    start_date = pd.to_datetime("2017-08-16")
    for i in range(-16, 0):
        for j in range(yhat.shape[1]):
            ans_df = pd.concat((ans_df, pd.DataFrame({"store_nbr": [int(y.item())], "family": [f_dict[j]], "sale": [yhat[i][j].item()], "date": [start_date]})))
        start_date += pd.Timedelta(days=1)

### recover the sales

In [47]:
cols = ["store_nbr", "family", "date"]
ans_df.sort_values(cols, inplace=True)
base_sales = sd.base_sales.sort_values(cols[:2]).set_index(cols[:2])
ans_out_df = pd.DataFrame()

date_lst = [] 
for i in range(16, 32):
    date_lst.append(pd.to_datetime(f"2017-08-{i}"))
    
for i in range(0, len(ans_df), 16):
    row = ans_df.iloc[i]
    store_nbr, family = row["store_nbr"], row["family"]
    base_s = base_sales.loc[(store_nbr, family)].sales
    sales_hat = Sales_Dataset.log_ret_2_sales(ans_df.iloc[i:i+16].sale, base_s)
    ans_out_df = pd.concat((ans_out_df, pd.DataFrame({"sales": sales_hat, "family": family, "store_nbr": store_nbr, "date": date_lst})))
# ans_out_df.sales = ans_out_df.sales.round().astype(int)
ans_out_df = ans_out_df.reset_index().set_index(["date", "store_nbr", "family"]).sort_index()

### output the answer

In [51]:
test_df = pd.read_csv(join(SALE_HOME, "test.csv"), index_col=0)
test_out = pd.DataFrame()
for id, (date, store_nbr, family, onpromotion) in test_df.iterrows():
    test_out = pd.concat((test_out, pd.DataFrame({"sales": ans_out_df.loc[(
        date, store_nbr, family
    )].sales}, index=[id])))
test_out

,sales
3000888,3.483087
3000889,0.000000
3000890,3.834394
3000891,1396.264388
3000892,0.000000
...,...
3029395,617.017298
3029396,193.074860
3029397,558.420662
3029398,7949.364318


In [56]:
test_out.index.name = "id"
test_out.to_csv(join(MODEL_HOME, "answer.csv"), index=True)